<a href="https://colab.research.google.com/github/pipeton8/smog-checks/blob/main/SMOG_Checks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SMOG Checks brand and model database cleanup
*(c) Felipe del Canto (PUC-Chile)*


This script is designed to recognize matching brands and models of cars from a Chilean fleet database.

Both models and brands are written differently in the base because of typos (i.e. NISSSAN instead of NISSAN), abbreviations (i.e. KIA instead of KIA MOTORS) or mispellings (i.e. SSANG YON instead of SSANGYON).

The script implements a comparison between each pair of strings in each database, based on the Jaro distance ([link](https://rosettacode.org/wiki/Jaro_distance)). This metric was chosen since it is does not penalize typos and small misspellings as much as the Hamming distance; and is inmediatly normalized between 0 and 1.

The output of this script is a txt file containing replace commands for the Stata statistical software. The goal is to make brand and model strings uniform throughout the database. 

## Preamble

In [ ]:
# Import files
!wget 'https://docs.google.com/uc?export=download&id=1RnADjbJK5rM4RYsHiFAoxQrBp8atbdw-' -O 'brandsDB.txt'
!wget 'https://docs.google.com/uc?export=download&id=1oEzvSa4fRezWCCH6EB004CUo80F9h1Vf' -O 'modelsDB.txt'
!mkdir 'Model Cleanup'

In [ ]:
# Install Jaro distance library
!pip install jellyfish

In [ ]:
# Load packages
import time
import os
import pandas as pd
import numpy as np
import jellyfish
import math

# Parameters and functions

## String functions

In [ ]:
# Longest word in the string
def longestWord(s):
  """This function takes the longest word in a several-word string and returns it."""
  words = s.split()
  maxLength = 0
  currentLongest =  ''

  for word in words:
    if len(word) > maxLength:
      maxLength = len(word)
      currentLongest = word
  
  return currentLongest

# Normalize strings
def normalizeString(s, method):
  """ This function "normalizes" a string by keeping only the alphanumeric characters
  and spaces and then following 3 possible methods:
  
    if method == "alphanumeric":
      The spaces are removed and the resulting string returned.
    
    elif method == "longest word":
      The longest word is returned, using the function longestWord.

    elif method == "first n", where n is an integer:
      The first n characters of the string are returned

  In any other case, a ValueError is raised.
  """

  s = ''.join(e for e in s.strip() if e.isalnum() or e == " ")

  if method == "alphanumeric":
    return ''.join(e for e in s if not(e == " "))

  elif method == "longest word":
    return longestWord(s)

  elif method.find("first") != -1:
    size = int(method.split()[-1])
    s = ''.join(e for e in s if not(e == " "))
    return s[:size]

  else:
    raise ValueError("Method not implemented")

## Jaro distance functions

In [ ]:
# Jaro distance with a lower bound
def jaroDistancesUpToP(x, y,p=0.8):
  """This function compute the Jaro distance between two strings and returns it if the result is greater or equal to p. Otherwise, it returns 0."""
  return jellyfish.jaro_distance(x, y) * (jellyfish.jaro_distance(x, y) >= p)

# Matches using the Jaro distance
def bestMatchesUpToP(x, y,p=0.8):
  """This function returns True if the strings have a jaro distance of at least p and False otherwise."""
  return jellyfish.jaro_distance(x, y) >= p

# Vectorize the functions to compute distances faster
jaroVectorized = np.vectorize(jaroDistancesUpToP)
matchVectorized = np.vectorize(bestMatchesUpToP)

## Dictionary functions

In [ ]:
# Sort dictionary
def sortDict(dictToSort):
  """ This functions sorts the given dictionary according to the length of its items, in descending order."""
  sortedDict = sorted(dictToSort.items(), key = lambda i : len(i[1]), reverse = True)

  return dict((x,y) for x,y in sortedDict)

# Get matches from a dictionary
def getMatchesDict(matches, names):
  """ This functions receives a binary array that indicates which elements are a match (1) or not (0). 
  Then, it returns a dictionary for which every key is a brand or model indicated in 'names' and every item
  is a list of the matches of this brand or model."""
  N = len(names)
  
  matchesDict = {}

  for i in range(N):
    matchIndex = np.argwhere(matches[i,:] == 1).flatten()
   
    if len(matchIndex) != 0:
      matchesDict[names[i]] = [names[j] for j in matchIndex]
    
  return sortDict(matchesDict)

## Output functions

In [ ]:
# Write replace commands
def writeReplaceContent(variable0,cluster,character,length,document,section="",variable1="",key1="",close = True):
  """ This function writes the replace comands for Stata. For the model cleanup
  process the variable1 and key1 variables are the brand and the section is the
  name of the model."""
  
  # Obtain keys from the match dictionary. Cluster is the dictionary that contains
  # a brand or all the models of a brand 
  keys = list(cluster.keys())

  # For every key, write the replaces
  for key in keys:

    # Start with the number of the section
    if section != "":
      # For models, the sectionNumber is section-subsection
      sectionNumber = section+"-"+str(keys.index(key)+1)
    else:
      # For brands, the sectionNumber is the number of
      sectionNumber = str(keys.index(key)+1)

    # Write the title
    writeTitle(key,sectionNumber,character,length,document)
    
    # Write the commands
    writeReplaces(variable0,key,cluster[key],document,variable1,key1)

  # Close the document if asked to
  if close:
    document.close()

  return None

# Write the title
def writeTitle(sectionTitle,sectionNumber,character,length,document):
    """ This function writes the title for each section of commands.
    It uses a certain 'character' to enclose the title."""
    
    # Top decoration
    document.write("".join(character for i in range(length)) + "\n")

    # Title
    document.write("{}) {}\n".format(sectionNumber,sectionTitle))

    # Bottom decoration
    document.write("".join(character for i in range(length)) + "\n")

    document.write("\n")

# Write each command
def writeReplaces(variable0,key0,cluster,document,variable1="",key1=""):
  """ This function writes each replace command. For the model cleanup process,
  variable1 and key1 are the brand variable name and its value, respectively."""
  
  # Cluster is a list of matches for key0
  for key2 in cluster:
    if variable1 != "":
      # This is intended for model cleanup (key1 is the brand name)
      document.write('replace {} = "{}" if {} == "{}" & {} == "{}"\n'.format(variable0,key0,variable1,key1,variable0,key2))
    else:
      # This is intended for brand cleanup
      document.write('replace {} = "{}" if {} == "{}"\n'.format(variable0,key0,variable0,key2))
  
  document.write("\n")

# Main code

## Brand cleanup function

In [ ]:
def matchByBrand(filePath,docTitle, distance = 0.8, method = 'alphanumeric', demo = False):
  """ This functions computes the matches by brand and writes the output document
  for the brand cleanup process. """

  if filePath == "" or docTitle == "":
    return None

  # Read database
  cars = pd.read_csv(filePath, keep_default_na=False)

  # If demo, sample 50% of the database at random
  if demo:
    cars = cars.sample(frac=0.5, random_state=0)

  # Normalize 'nombre_marca' and take the first and the last 5 characters
  cars['nombreMarcaNormalized'] = cars['nombre_marca'].apply(normalizeString, args = (method,)).str.upper()

  # Drop observations whose nombreMarcaNormalized is '' and reset index
  cars = cars.loc[cars['nombreMarcaNormalized'] != '']
  cars.reset_index(drop = True, inplace=True)

  # Compute matches
  print("Beginning with Normalized Brands...", end="")
  matchesByBrand = matchVectorized(np.array(cars['nombreMarcaNormalized'])[:,np.newaxis], np.array(cars['nombreMarcaNormalized'][:]),p=distance)
  print("Done!")

  # Get match dictionaries
  clustersByBrandDict = getMatchesDict(matchesByBrand,cars['nombre_marca'])

  # Create document
  documentByBrand = open(docTitle, "w")

  # Write them
  print("Writing document...", end="")
  writeReplaceContent('nombre_marca', clustersByBrandDict,'*',50, documentByBrand)
  print("Done!")

  return None


## Model cleanup function

In [ ]:
def matchByModel(filePath,docPath, distance = 0.8, minCount = 1000, method = "alphanumeric", demo = False):
  """ This functions computes the matches by model for different brands and 
  writes the output document for the model cleanup process. """

  if filePath == "" or docPath == "":
    return None

  # Read database
  cars = pd.read_csv(filePath, keep_default_na=False)

  # If demo, sample 10% of the database at random
  if demo:
    cars = cars.sample(frac=0.1,random_state=0)

  # Convert brands to uppercase letters
  cars['nombre_marca']=cars['nombre_marca'].str.upper()
  
  # Keep brands that contain at least minCount cars
  brandCount = cars.pivot_table(index=['nombre_marca'], aggfunc='size')
  brandsToWork = brandCount[(brandCount >= minCount)].sort_values(ascending=False)

  # Normalize 'nombre_modelo'
  cars['nombreModeloNormalized'] = cars['nombre_modelo'].apply(normalizeString, args = (method,)).str.upper()

  # Drop observations whose nombreModeloNormalized is '' and reset index
  cars = cars.loc[cars['nombreModeloNormalized'] != '']
  cars.reset_index(drop = True, inplace=True)

  for brand in brandsToWork.keys():
    print("")
    print("Starting with " + brand)
    
    # Get cars of brand
    carsOfBrand = cars.loc[cars['nombre_marca'] == brand]
    carsOfBrand.reset_index(drop = True, inplace=True)

    # Compute matches
    print("- Computing Matches...", end="")
    matchesByModel = matchVectorized(np.array(carsOfBrand['nombreModeloNormalized'])[:,np.newaxis], np.array(carsOfBrand['nombreModeloNormalized'][:]),p=distance)
    print("Done!")

    # Get match dictionaries
    print("- Transforming to dictionary...", end="")
    clustersByModelDict = getMatchesDict(matchesByModel,carsOfBrand['nombre_modelo'])
    print("Done!")

    # Create document
    print("- Writing document...", end="")
    sectionNumber = str(list(brandsToWork.keys()).index(brand)+1)
    docTitle = sectionNumber + " - " + brand + ".txt"
    brandDocument = open(docPath+docTitle, "w")

    # Write document
    writeTitle(brand,sectionNumber,'*',50, brandDocument)
    writeReplaceContent('nombre_modelo', clustersByModelDict,'-',30, brandDocument, section=sectionNumber, variable1='nombre_marca', key1=brand)
    print("Done!" + "\n")

    brandDocument.close()

## Execution
Expected run time in demo mode: 2 min

In [ ]:
# Obtain matches and write documents
matchByBrand("brandsDB.txt","brandsCleanup.txt", distance = 0.8, method = 'alphanumeric', demo = True)
matchByModel("modelsDB.txt","Model Cleanup/", distance = 0.8, minCount = 1000, method = 'longest word', demo = True)
#matchByModel("modelsDB.txt","Model Cleanup/", distance = 0.8, minCount = 1000, method = "first 3", demo = True)

print("I'm done!")